# 6장 데이터 로딩과 저장, 파일 형식

In [1]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
pd.options.display.max_colwidth = 75
pd.options.display.max_columns = 20
np.set_printoptions(precision=4, suppress=True)
plt.rcParams["font.family"] = 'Malgun Gothic'

# 1. csv 텍스트 데이터
## 1.1 csv파일 읽기
* 판다스에서 CSV 파일을 읽기: pd.read_csv(path, encoding = 'utf-8') 함수를 사용(파일명, 영어나 한국어)
    * path 인수 : 파일시스템에서의 위치, URL, 파일객체를 나타내는 문자열
    * encoding 인수 - 유니코드 인코딩 종류을 지정


    1) header(컬럼명)가 있는 파일

In [2]:
!cat examples/ex1.csv 
df = pd.read_csv("examples/ex1.csv") #csv 파일을 -> DataFrame으로 반환
type(df)
df.head(2) # 맨 위의 3줄

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world


    2) header가 없는 파일 : 컬럼을 지정
        * names 인수 - 열 이름 리스트
        * header 인수 - 헤더가 없을 경우 None으로 지정

In [3]:
!cat examples/ex2.csv
# pd.read_csv("examples/ex2.csv")

names = ["a", "b", "c", "d", "message"]
data = pd.read_csv("examples/ex2.csv", names=names) # names 인수 - 데이터프레임의 컬럼명 지정
                                                    # 헤더가 없으면 지정해주기
data

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


    3) 계층적 색인 지정하기
        * index_col 인수 - 색인으로 사용할 열 번호나 이름, 계층적 색인을 지정할 경우 리스트

In [4]:
!cat examples/csv_mindex.csv
parsed = pd.read_csv("examples/csv_mindex.csv",
                     index_col=["key1", "key2"]) # 특정 컬럼을 지정해서 섹인함
parsed

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

    4) 여러 개의 공백(spacebar)으로 필드를 구분한 파일 txt포멧임
        * sep 인수 - 필드을 구분하기 위해 사용할 정규 표현식

In [5]:
!cat examples/ex3.txt

result = pd.read_csv("examples/ex3.txt", sep="\s+") # 정규 표현식 : "  \s+  " +는 여러걔의 공백
result

            A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491


,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


    5) 주석을 갖고 있는 파일
        * skiprows 인수 - 파일의 시작부터 무시할 행 수 또는 무시할 행 번호가 담긴 리스트
        * 주석을 포함한 무시하고 싶은 값을 의미함 -> 2개 이상의 값은 앵간하면 리스트에 담기

In [6]:
!cat examples/ex4.csv
pd.read_csv("examples/ex4.csv", skiprows=[0, 2, 3]) # -> 이것 또한 리스트에 담아서 처리

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


    6) 누락된 값이 있는 파일 : 비어 있는 문자열, NA, NULL을 NaN으로 출력 
        * na_values 인수 - NA 값으로 처리할 값들의 목록 
        * 원본에 있는 값들 중 개발자가 null처리 한 값의 목록

In [7]:
!cat examples/ex5.csv
# na_values 인수를 사용하지 않은 경우 
result = pd.read_csv("examples/ex5.csv")
result
# 1) 누락된 값을 확인 : isna()함수 이용, isnull(), natna(), natnull()
pd.isna(result)

# # 2) na_values -> 컬럼별 문자열 집합을 받아서 누락된 값 처리
sentinels = {"message": ["foo", "NA"], "something": ["two"]}
result = pd.read_csv("examples/ex5.csv", na_values=sentinels)
print(result)
result.isna()

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo
  something  a   b     c   d message
0       one  1   2   3.0   4     NaN
1       NaN  5   6   NaN   8   world
2     three  9  10  11.0  12     NaN


,something,a,b,c,d,message
0,False,False,False,False,False,True
1,True,False,False,True,False,False
2,False,False,False,False,False,True


    7) 텍스트 파일 조금씩 
        * nrows 인수 - 파일의 첫 일부만 읽어올 때 처음 몇 줄을 읽을 것인지 지정 
        * chunksize 인수 - TextParser 객체에서 사용할 한 번에 익을 파일의 크기

In [8]:
pd.options.display.max_rows = 10 # -> 중간을 생략하고 10개의 행을 가져옴다
# !cat examples/ex6.csv
# result = pd.read_csv("examples/ex6.csv")
# result
# result = pd.read_csv("examples/ex6.csv", nrows=5) # 몇번 읽을지
# result

### [예제]  ex6.csv 파일을 순회하면서 "key"열의 값을 세기

In [9]:
chunker = pd.read_csv("examples/ex6.csv", chunksize=1000) # 청크 사이즈를 이용한 중복되는 키 값 찾기
# type(chunker) -> pandas.io.parsers.readers.TextFileReader 한 행식 가져와야함
# 1)리스트의 시리즈 생성(리스트, 딕셔널이를 사용해서 시리즈 를 만들 수 있음) -> 청크 단위로 key 열의 값을 저장
tot = pd.Series([], dtype='int64')
type(tot) # 시리즈임
for piece in chunker:
    # print(type(piece["key"]))
    # print(piece["key"].value_counts()) # 각각의 키 값이 몇번 나왔는지
    tot = tot.add(piece["key"].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False) #각각의 값을 내림차순으로 저장
tot
tot[:10]

key
E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

In [10]:
from pandas import Series
tot = pd.Series([], dtype='int64')
for piece in range(3):
    tot = tot.add(Series([1,2,3]), fill_value = 10)
tot

0    13.0
1    16.0
2    19.0
dtype: float64

## 1.2. csv 텍스트 파일 저장하기
* 판다스에서 CSV 파일을 저장하기: df.to_csv(path, encoding = 'utf-8') 함수를 사용 인코딩 생략 해도됨
    * path 인수 : 파일시스템에서의 위치, URL, 파일객체를 나타내는 문자열
    * encoding 인수 - 유니코드 인코딩 종류을 지정


In [11]:
#1) 준비 :DataFrame
!cat examples/ex5.csv
data = pd.read_csv("examples/ex5.csv") # -> dataFrame으로 변경'
#2) 조작 : dataFrame에서 indexing을 해서 필요한 데이터 추출 -> 리스트에 담아줘야함 -> DataFrmame
data1 = data[['a','b','d']]
data1
#3) dataq -> csv 파일로 저장 
data1.to_csv("examples/out.csv")

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo


In [12]:
data.to_csv("examples/out.csv", sep="|") # 구분자 사용
# data.to_csv("examples/out.csv", na_rep = "NULL") #누락된 값을 원하는 값으로 지정
# data.to_csv("examples/out.csv", index=False, header=False)
# data.to_csv("examples/out.csv", index=False, columns=["a", "b", "c"])
!cat examples/out.csv
!


|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


# 2. JSON  데이터
* JSON(JavaScript Object Notation)은 텍스트 데이터를 저장하고 전송하기 위한 일반적인 형식
* 주로 웹에서 데이터를 전송하거나 저장하는 데 사용
* JSON 파일은 일반적으로 텍스트 형식으로 작성되며, 사람과 기계 모두 이해하기 쉽다.
* JSON 파일은 키-값 쌍의 모음으로 구성
  * 키-값 쌍은 객체(object)라고도 하며, 중괄호 {} 로 묶는다.
  * 배열(array)은 대괄호 [] 안에 값의 목록을 나열하여 정의.

## 2.1 JSON 데이터 읽기/저장
    * pandas의 to_json() -> 데이터프레임을 JSON 파일로 저
    * read_json() -> JSON 파일에서 데이터프레임을 읽어오기

### [예제]  pandas의 to_json() 및 read_json() 메서드를 사용하여 데이터프레임을 JSON 파일로 저장하고 읽기

In [10]:
# 1. pandas 라이브러리 임포트

import pandas as pd

# 2. JSON 파일로 저장할 데이터프레임 생성
data = {
    "name": ["John", "Alice", "Bob"],
    "age": [30, 25, 35],
    "city": ["New York", "Los Angeles", "Chicago"]
} # -> 딕셔너리 파일
df = pd.DataFrame(data)

# 3. 데이터프레임을 JSON 파일로 저장
df.to_json("examples/data.json")

# JSON 파일을 읽어와 데이터프레임으로 변환
df_read = pd.read_json("examples/data.json")

print("Original DataFrame:")
print(df)

print("\nDataFrame from JSON file:")
print(df_read)


Original DataFrame:
    name  age         city
0   John   30     New York
1  Alice   25  Los Angeles
2    Bob   35      Chicago

DataFrame from JSON file:
    name  age         city
0   John   30     New York
1  Alice   25  Los Angeles
2    Bob   35      Chicago


# 3. HTML 데이터
## 3.1 웹 스크래핑 - read_html()

* read_html() - html 문서 내 테이블 데이터 읽어 테이블 구조의 모든 데이터를 데이터프레임으로 저장해서 반환
    * https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list/


#### [예제] 미국 예금 보험 공사에서 부도 은행을 보여주는 HTML 문서 읽어오기

### 개발자 도구 사용법
* #s_content > div.section > ul > li:nth-child(1) > dl > dt > a
* div를 이용해서 독립적으로 만듦

In [24]:
import pandas as pd
from pandas import Series, DataFrame

#tables = pd.read_html("examples/fdic_failed_bank_list.html") 
tables = pd.read_html("https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list/")
len(tables)
print(type(tables))
failures = tables[0]
print(type(failures))
failures.head()

<class 'list'>
<class 'pandas.core.frame.DataFrame'>


,Bank NameBank,CityCity,StateSt,CertCert,Acquiring InstitutionAI,Closing DateClosing,FundFund
0,Citizens Bank,Sac City,IA,8758,Iowa Trust & Savings Bank,"November 3, 2023",10545
1,Heartland Tri-State Bank,Elkhart,KS,25851,"Dream First Bank, N.A.","July 28, 2023",10544
2,First Republic Bank,San Francisco,CA,59017,"JPMorgan Chase Bank, N.A.","May 1, 2023",10543
3,Signature Bank,New York,NY,57053,"Flagstar Bank, N.A.","March 12, 2023",10540
4,Silicon Valley Bank,Santa Clara,CA,24735,First–Citizens Bank & Trust Company,"March 10, 2023",10539


In [25]:
# 1) 컬럼명 수정하기
failures.columns
failures.columns = ['BankNameBank', 'CityCity', 'StateSt', 'CertCert',
       'AcquiringInstitutionAI', 'ClosingDate', 'FundFund']
failures.head(3)

,BankNameBank,CityCity,StateSt,CertCert,AcquiringInstitutionAI,ClosingDate,FundFund
0,Citizens Bank,Sac City,IA,8758,Iowa Trust & Savings Bank,"November 3, 2023",10545
1,Heartland Tri-State Bank,Elkhart,KS,25851,"Dream First Bank, N.A.","July 28, 2023",10544
2,First Republic Bank,San Francisco,CA,59017,"JPMorgan Chase Bank, N.A.","May 1, 2023",10543


In [26]:
# 2) 데이터 변환 : 문자열 -> datetime
close_timestamps = pd.to_datetime(failures["ClosingDate"])# 컬럽 인덱싱
close_timestamps
#3) 년도별 데이터 개수 추출
close_timestamps.dt.year.value_counts() #Series/ value_counts() 동일한 값이 몇번 나왔는지

ClosingDate
2010    157
2009    140
2011     92
2012     51
2008     25
2013     24
2014     18
2002     11
2017      8
2015      8
2023      5
2016      5
2020      4
2004      4
2019      4
2001      4
2007      3
2003      3
2000      2
Name: count, dtype: int64

#### [실습] : 네이버 주식 데이터 가져오기 - read_html()
* https://www.naver.com/ -> 증권 -> 국내증시 -> 시가총액
    * https://finance.naver.com/sise/sise_market_sum.naver

In [33]:
!pip install html5lib  lxml


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import pandas as pd

# 'euc-kr' 또는 'cp949'로 인코딩 설정하여 페이지 읽기 시도
df_list = pd.read_html("https://finance.naver.com/sise/sise_market_sum.naver",encoding='cp949')
type(df_list)
len(df_list)
type(df_list[1])
# 데이터프레임 확인
for df in df_list:
    print("\n", df.head())


C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\bs4\__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")



      0      1          2        3          4       5
0  거래량   매수호가   거래대금(백만)  시가총액(억)    영업이익(억)  PER(배)
1   시가   매도호가      전일거래량  자산총계(억)    영업이익증가율  ROE(%)
2   고가  매수총잔량      외국인비율  부채총계(억)   당기순이익(억)  ROA(%)
3   저가  매도총잔량  상장주식수(천주)   매출액(억)   주당순이익(원)  PBR(배)
4  NaN    NaN        NaN   매출액증가율  보통주배당금(원)  유보율(%)

      N       종목명       현재가      전일비     등락률     액면가       시가총액      상장주식수  \
0  NaN       NaN       NaN      NaN     NaN     NaN        NaN        NaN   
1  1.0      삼성전자   84100.0    900.0  -1.06%   100.0  5020587.0  5969783.0   
2  2.0    SK하이닉스  179200.0   7100.0  -3.81%  5000.0  1304580.0   728002.0   
3  3.0  LG에너지솔루션  376000.0  17000.0  -4.33%   500.0   879840.0   234000.0   
4  4.0  삼성바이오로직스  812000.0   4000.0  -0.49%  2500.0   577933.0    71174.0   

   외국인비율         거래량    PER    ROE  토론실  
0    NaN         NaN    NaN    NaN  NaN  
1  55.57  30136601.0  39.47   4.15  NaN  
2  54.69   5082907.0 -14.32 -15.61  NaN  
3   4.97    364327.0  71.12   6.36  NaN  
4  11

In [10]:
# 데이터프레임 탐색 종목명에 NaN이 있는 행은 제거
df = df_list[1]
df.head(10)
#1. boolean indexing
df["종목명"] #1. 컬럼 추출 - Series
df["종목명"].notnull() # 조건 색인
data = df[df['종목명'].notnull()] #3. 조건색인을 넣어주기
data
data.head()
data.to_csv("Naver_kospi.csv")
data

,N,종목명,현재가,전일비,등락률,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE,토론실
1,1.0,삼성전자,84100.0,900.0,-1.06%,100.0,5020587.0,5969783.0,55.57,30136601.0,39.47,4.15,NaN
2,2.0,SK하이닉스,179200.0,7100.0,-3.81%,5000.0,1304580.0,728002.0,54.69,5082907.0,-14.32,-15.61,NaN
3,3.0,LG에너지솔루션,376000.0,17000.0,-4.33%,500.0,879840.0,234000.0,4.97,364327.0,71.12,6.36,NaN
4,4.0,삼성바이오로직스,812000.0,4000.0,-0.49%,2500.0,577933.0,71174.0,11.99,50553.0,67.38,9.12,NaN
5,5.0,삼성전자우,68300.0,1600.0,-2.29%,100.0,562032.0,822887.0,75.60,1916769.0,32.05,NaN,NaN
9,6.0,현대차,216000.0,4000.0,-1.82%,5000.0,456908.0,211532.0,38.16,1097532.0,4.96,13.68,NaN
10,7.0,기아,101800.0,2900.0,-2.77%,5000.0,409281.0,402044.0,40.33,1658452.0,4.68,20.44,NaN
11,8.0,셀트리온,180500.0,3300.0,-1.80%,1000.0,393580.0,218050.0,23.02,472146.0,49.60,5.07,NaN
12,9.0,POSCO홀딩스,398500.0,11000.0,-2.69%,5000.0,337016.0,84571.0,27.26,711528.0,19.85,3.18,NaN
13,10.0,NAVER,193400.0,1400.0,+0.73%,100.0,314098.0,162409.0,45.52,1095304.0,31.29,4.41,NaN


## 3.2 웹 스크래핑 - requests

* 웹 페이지 정보 가져오기 - requests 패키지 이용
  * requests.get()
  * requests.get().status_code
  * requests.get().text


### [예제] 타겟 사이트의 json 구조로 저장된 데이터 가져오기

In [13]:
!pip install requests


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [36]:
# 1. requests 모듈 설치하기
# !pip install requests
# 2. requests 모듈 임포트하기
import requests
import pandas as pd 
#3. url 가져오기
url = "https://api.github.com/repos/pandas-dev/pandas/issues"
# pd.read_html(url) -> No tables found
# json 페이지 가져오기
resp = requests.get(url)
# type(resp) -> Response
#print(resp.status_code)
resp.text
type(resp.text) # -> 스트링문자열임

# # 4. json 내용을 딕셔너리로 변환
data = resp.json() # json의 내용을 딕셔너리 형태로 변환한 객체을 반환
type(data) # dick형 맞으면 200 출력
data


{'message': "API rate limit exceeded for 58.233.133.18. (But here's the good news: Authenticated requests get a higher rate limit. Check out the documentation for more details.)",
 'documentation_url': 'https://docs.github.com/rest/overview/resources-in-the-rest-api#rate-limiting'}

### [문제] 네이버 블로그의 키워드 관련 검색 결과 텍스트 데이터 가져오기 - requests 모듈 사용시 한계점

In [45]:
import requests
# url = "https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=ALL&orderBy=sim&keyword=%ED%8C%8C%EC%9D%B4%EC%8D%AC"

params = {
    'pageNo' : 1,
    'rangeType' : 'ALL',
   'orderBy' : 'sim',
    'keyword' : '파이썬'
}
#pd.read_html(url)

response = requests.get('https://section.blog.naver.com/Search/Post.naver?', params=params)

print(response.status_code) # HTTP GET 방식으로 URL 파라미터 값 전달
print(response.url) # status_code 는 응답코드
print(response.text) # text에는 HTML 코드

200
https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=ALL&orderBy=sim&keyword=%ED%8C%8C%EC%9D%B4%EC%8D%AC




<!DOCTYPE html>
<html lang="ko">
<head>
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <base href="/home" />
    <meta name="robots" content="noindex,nofollow"/>
    <meta name="referrer" content="always"/>
    <meta name="format-detection" content="telephone=no">
    <link rel="shortcut icon" type="image/x-icon" href="https://section.blog.naver.com/favicon.ico?3"/>
    




 

	
		<meta property="og:title" content="파이썬 : 네이버 블로그"/>

		
			
			
			<meta property="og:image" content="https://blogimgs.pstatic.net/nblog/mylog/post/og_default_image_160610.png"/>
			
		
        <meta property="og:description" content="당신의 모든 기록을 담는 공간"/>

		
	
	

<meta property="me:feed:serviceId" content="blog" />

    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
    <title>네이버 블로그</title>

    <link rel="stylesheet" type="text/css" href="htt

## 3.3 웹 스크래핑 - beautifulsoup
* 웹 페이지 정보 추출하기 - beautifulsoup 사용법 
  * soup.select_one()
  * soup.select()
  * get_text()
* 개발자 도구 사용

### [문제] 네이버 지식iN의 검색어 관련 검색 결과 제목 가져오기
* soup.select_one() -> 한개
* soup.select() -> 여러개
*  개발자 도구를 이용 (마우스 우클릭 해서 검사 누르기)

In [15]:
import requests
from bs4 import BeautifulSoup
# 1 .타켓 웹사이트 url
url = 'https://kin.naver.com/search/list.naver?query=%ED%8C%8C%EC%9D%B4%EC%8D%AC'
# 2. 웹페이지 로딩
response = requests.get(url)

if response.status_code == 200:
    # 3. 웹페이지 확인 200이 아니면 else로 감
    html = response.text
    # 4. html -> beautifulsoup 객체로 변환: 원하는 데이터를 출력(그냥은 출력이 안됨)
    soup = BeautifulSoup(html, 'html.parser')

    # 5. 테이터 추출 : 첫번째 제목 가져오기
    #s_content > div.section > ul > li:nth-child(1) > dl > dt > a
    title = soup.select_one('#s_content > div.section > ul > li:nth-child(1) > dl > dt > a')
    # print(title)
    print(title.get_text()) # -> 텍스트만 추출

    # # 6. 데이터 추출 : 선택해서 여러개의 제목 가져오기
    # titles = soup.select('#s_content > div.section > ul > li:nth-child(n) > dl > dt > a')
    # for title in titles:
    #     print(title.get_text())
else : 
    print(response.status_code)



파이썬이 미래에 도움이 될까요?


## 3.4 웹 스크래핑 - Selenium을 이용하여 사이트 자동화하기(클로러 하기)
(다음주 수업 비대면 )
* selenium 사용법
  1. 시스템의 OS 및 Chrome 브라우저의 버전 확인
  2. Chrome 드라이버 다운로드 및 설치
  3. 크롬 드라이버을 프로젝트 폴더에 저장 - 크롬 드라이버의 설치 경로
     * from selenium import webdriver
     * driver = webdriver.Chrome()
* 동적으로 웹 페이지 정보 추출하기
     * driver.get("웹주소") 
  

### [예제]

In [16]:
!pip install selenium

   ---------------------------------------- 0.0/10.5 MB ? eta -:--:--
   -- ------------------------------------- 0.8/10.5 MB 24.8 MB/s eta 0:00:01
   ------ --------------------------------- 1.8/10.5 MB 22.6 MB/s eta 0:00:01
   ------- -------------------------------- 2.1/10.5 MB 22.1 MB/s eta 0:00:01
   --------- ------------------------------ 2.6/10.5 MB 15.0 MB/s eta 0:00:01
   ------------- -------------------------- 3.6/10.5 MB 16.5 MB/s eta 0:00:01
   ----------------- ---------------------- 4.6/10.5 MB 17.4 MB/s eta 0:00:01
   -------------------- ------------------- 5.4/10.5 MB 17.4 MB/s eta 0:00:01
   ------------------------ --------------- 6.5/10.5 MB 18.7 MB/s eta 0:00:01
   ---------------------------- ----------- 7.5/10.5 MB 19.1 MB/s eta 0:00:01
   -------------------------------- ------- 8.5/10.5 MB 19.4 MB/s eta 0:00:01
   ------------------------------------ --- 9.5/10.5 MB 19.6 MB/s eta 0:00:01
   ---------------------------------------  10.5/10.5 MB 19.9 MB/s eta 0

In [22]:
from selenium import webdriver # -> 클로러를 만들떄 사용하는 import
# 1. Chrome 브라우저 실행
driver = webdriver.Chrome() # 크롬 드라이버 실행파일의 위치를 넣어줘야한당 - 위치는 작업 디렉토리에 있다
# 타겟의 웹페이지 로딩
url = "httpss://www.google.com"
driver.get(url)
# 3. 타겟 브라우저 종료 
driver.quit() # .quit -> 종료 코딩

### [문제]
* driver.find_elementS() 테그 찾기
* .send_keys -> 엔터키 전달


In [27]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

# 1 Chrome 브라우저 실행
driver = webdriver.Chrome()

#  2 google 홈페이지 열기
url = "https://www.google.com"
driver.get(url)

# 3 페이지가 완전히 로드될 때까지 2초 대기
time.sleep(2) # 파이선 기본 라이브러리
#  4 검색기능 - 검색창 검색어 입력 엔터키 클릭
try:
    # 검색 상자 요소 찾기 -> 테그 찾기
    search_box = driver.find_element(By.NAME, "q")

    # 검색어 입력 -> 고정되지 않음 대소문자 구분 없음
    search_query = "OpenAI"
    search_box.send_keys(search_query) #-> 검색어 넣기 

    # Enter 키를 눌러 검색 수행
    search_box.send_keys(Keys.RETURN) # 엔터키 시행 코드

    # 검색 결과가 로드될 때까지 5초 대기
    time.sleep(5) # -> 검색 후 대기 기간

    # 검색 결과 추출
    search_results = driver.find_elements(By.CSS_SELECTOR, "div.g")
    # 검색 결과의 제목과 URL 인쇄
    # for result in search_results:
    #     title = result.find_element(By.CSS_SELECTOR, "h3").text
    #     url = result.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
    #     print("제목:", title)
    #     print("URL:", url)
    #     print()
    search_results = driver.find_elements(By.CSS_SELECTOR, "div.g") 
    title = search_results[0].find_element(By.CSS_SELECTOR, "h3").text
    url = search_results[0].find_element(By.CSS_SELECTOR, "a").get_attribute("href")
    print("제목:", title)
    print("URL:", url)

except Exception as e:
    print("오류가 발생했습니다:", e)

finally:
# 브라우저 창 닫기
     driver.quit()


제목: OpenAI
URL: https://openai.com/


### [문제] Selenium을 이용하여 네이버 증권 웹페이지 열기/ 삼성전자 일별 시세 HTML가져오기/ 10페이지 내역 테이블 데이터 찾기/검색 결과를 데이터프레임으로 변환/저장하기
* 타겟 사이트: https://finance.naver.com/item/sise.naver?code=005930* 

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

In [5]:
#1. 크롬 드라이버 실행
driver= webdriver.Chrome()

# 2페이지 로딩 시간
time.sleep(5)

# 3. 크롤링 사이트 : 삼성전자의 일별 시세 페이지 URL 찾기
## https://finance.naver.com/item/sise_day.naver?code=005930&page=1
samsung_url ="https://finance.naver.com/item/sise_day.naver?code=005930&page=" #base url


# # 4. 페이지 요청
# driver.get(samsum_url)

# 4. 첫 페이지부터 3 페이지까지 데이블 담기
samsung_daily_kospi = [] # 더이터를 담기위한 리스트 선언
for num in range(1,4):  # 시작하는 페이지가 1부터 1,2,3
    full_url = samsung_url + str(num)
    # 페이지 요청
    driver.get(full_url)
    
    try:
        # 페이지의 HTML 가져오기
        html = driver.page_source
        df = pd.read_html(html) # 테이터 프레임으로 변환
        print(df)
        # 결측치 처리
        ## 조건 색인
        cond = df[0]['날짜'].notnull() # 하나의 시리즈
        samsung_day = df[0][cond] # -> 결측치 처리가 끝난 값
        ## 페이지 별로 리스트에 담음
        samsung_daily_kospi.append(samsung_day)# -> 첫번째 페이지에 처리가 끝난 값을 인덱싱 
        
    except Exception as e:
        print("Network Error",e)
    # finally:
    #     driver.quit()

print(samsung_daily_kospi)
samsung_daily_kospi_all = pd.concat(samsung_daily_kospi) # 데이터 합치기 -> 리스트안에 데이터 프레임이기 떄문에 for문 x
samsung_daily_kospi_all.to_csv("examples/samsung_daily_kospi_2024_04_10.csv")


[            날짜       종가     전일비       시가       고가       저가         거래량
0          NaN      NaN     NaN      NaN      NaN      NaN         NaN
1   2024.04.09  83600.0   900.0  84500.0  84900.0  83100.0  18342168.0
2   2024.04.08  84500.0     0.0  85200.0  86000.0  84500.0  18953232.0
3   2024.04.05  84500.0   800.0  84500.0  85000.0  83800.0  18883752.0
4   2024.04.04  85300.0  1200.0  85200.0  85500.0  84300.0  25248934.0
5   2024.04.03  84100.0   900.0  84300.0  85000.0  83500.0  30493347.0
6          NaN      NaN     NaN      NaN      NaN      NaN         NaN
7          NaN      NaN     NaN      NaN      NaN      NaN         NaN
8          NaN      NaN     NaN      NaN      NaN      NaN         NaN
9   2024.04.02  85000.0  3000.0  82900.0  85000.0  82900.0  37077944.0
10  2024.04.01  82000.0   400.0  83200.0  83300.0  82000.0  20116513.0
11  2024.03.29  82400.0  1600.0  81200.0  82500.0  80900.0  27126366.0
12  2024.03.28  80800.0  1000.0  79400.0  81000.0  79200.0  25084812.0
13  2

C:\Users\Kimjung\AppData\Local\Temp\ipykernel_23980\597378474.py:25: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html) # 테이터 프레임으로 변환
C:\Users\Kimjung\AppData\Local\Temp\ipykernel_23980\597378474.py:25: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html) # 테이터 프레임으로 변환
C:\Users\Kimjung\AppData\Local\Temp\ipykernel_23980\597378474.py:25: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html) # 테이터 프레임으로 변환
